In [1]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from mne.stats import permutation_cluster_test
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization

### Add Directories/ Load Files

In [2]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


### Select Peaks and Plot Analytic Signal

In [4]:
#Load raw fif data

test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'with_med_FTG',
        'Sub065_ARTREJECT_FIF.fif'
    )
)

subID = 'Sub065'
fft_name = str(subID) + '_'
print(fft_name)

Opening raw data file C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\with_med_FTG\Sub065_ARTREJECT_FIF.fif...
    Reading extended channel information
    Range : 0 ... 113699 =      0.000 ...   454.796 secs
Ready.


C:\Users\mathiopv\AppData\Local\Temp\ipykernel_380\529687346.py:3: RuntimeWarning: This filename (C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT\data\Fifs\with_med_FTG\Sub065_ARTREJECT_FIF.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  test_raw = mne.io.read_raw_fif(os.path.join(


Sub065_


In [10]:
reload(dat_preproc)
x = test_raw.get_data() 
x1 = x[0,:]

peakMed = 80
peakStim = 63

dat_ngam = dat_preproc.low_highpass_filter(x1, peakMed-2, peakMed+2) 
dat_subh = dat_preproc.low_highpass_filter(x1, peakStim-2, peakStim+2) 

datall = [dat_ngam, dat_subh] 
labels = ['Peak'+str(peakMed)+'Hz','Peak'+str(peakStim)+'Hz']

#datall = [dat_subh] 
#labels = ['Peak'+str(peakStim)+'Hz']

print(labels)

['Peak80Hz', 'Peak63Hz']


In [6]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [11]:
%matplotlib qt
sm_signal_np = np.empty(shape = (2, x1.shape[0] - 499))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(1, 1, figsize=(12, 5))
wintosmooth = 500

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(np.squeeze(amplitude_envelope))

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    plt.plot(sm_signal, label = labels[idx])
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    #axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    #axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    plt.ylabel('Analytic Signal')
    plt.xlim([0, sm_signal.shape[0]])

    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 500 samples')
plt.legend(loc='upper right')

plt.show()


In [14]:
x2 = x[4, :] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim)

In [19]:
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,:], label = labels[idx], lw = 2)
ax2.plot(sm_stim1[:], label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
#ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
#ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
#ax1.set_xlim(0, sm_signal_np.shape[0])
#ax1.set_xticks(np.arange(0, 100000, 20000))
#ax1.set_xticklabels(np.arange(0, 400, 80))
ax1.set_xlabel('Time [samples]')
plt.title(str(subID))
plt.show()

In [17]:
sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]], [sm_signal_np[1]],[sm_stim1]])))
#sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]],[sm_stim1]])))
sm_analSignal_df = pd.DataFrame(sm_analSignal, 
    columns = ['Spontan','StimOn','StimVec'],
    )
print(sm_analSignal_df)

         Spontan    StimOn  StimVec
0       1.054341  0.562997      0.0
1       1.054652  0.562958      0.0
2       1.054256  0.564084      0.0
3       1.052994  0.565464      0.0
4       1.051952  0.566688      0.0
...          ...       ...      ...
113196  1.502762  0.820121      NaN
113197  1.503860  0.822183      NaN
113198  1.505077  0.824352      NaN
113199  1.505716  0.826316      NaN
113200  1.506881  0.828477      NaN

[113201 rows x 3 columns]


In [18]:
fft_fig = os.path.join(project_path, 'figures','anal_signal','with_med_FTG/')
fft_file = os.path.join(project_path, 'data','anal_signal','with_med_FTG/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalARTREJ',dpi = 300)
sm_analSignal_df.to_csv(str(fft_file)+str(fft_name)+'sm_analSignalARTREJ.csv')

### Crop Analytic Signal to Epochs of Interest

In [21]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'with_med_FTG',
        'Sub065_sm_analSignalARTREJ.csv'
))

print(sm_anal)

subID = 'Sub065'
fft_name = str(subID) + '_'
print(fft_name)

        Unnamed: 0   Spontan    StimOn  StimVec
0                0  1.054341  0.562997      0.0
1                1  1.054652  0.562958      0.0
2                2  1.054256  0.564084      0.0
3                3  1.052994  0.565464      0.0
4                4  1.051952  0.566688      0.0
...            ...       ...       ...      ...
113196      113196  1.502762  0.820121      NaN
113197      113197  1.503860  0.822183      NaN
113198      113198  1.505077  0.824352      NaN
113199      113199  1.505716  0.826316      NaN
113200      113200  1.506881  0.828477      NaN

[113201 rows x 4 columns]
Sub065_


In [ ]:
300000/250

In [145]:
%matplotlib qt
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
#ax1.plot(np.arange(0, sm_anal.shape[0]), sm_anal['Spontan'], label = 'Peak 81Hz')
ax1.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimOn'], label = 'Peak 65Hz')
ax2.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimVec'], ls='--', color = 'grey')

'''
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax1.set_xticklabels(np.arange(-200,1400,200))
ax1.set_xlabel('Time [sec]')
'''


"\nax1.legend()\nax1.set_ylabel('Z-scored Smoothed Analytic Signal')\nax2.set_ylabel('Stimulation Amplitude [mA]')\nax1.set_xticklabels(np.arange(-200,1400,200))\nax1.set_xlabel('Time [sec]')\n"

In [ ]:
20*250

In [23]:
#CROP IT

#med1 = 52
#med2 = 82
sub_pt = 110732
time_on = sub_pt - 20*250
time_off = sub_pt + 10*250

#sm_anal1 = sm_anal.iloc[med1*250 : med2*250]
sm_anal2 = sm_anal.iloc[time_on : time_off]
sm_toplot = pd.concat([sm_anal2])
print(sm_toplot)

        Unnamed: 0   Spontan    StimOn  StimVec
105732      105732  0.889422  1.226258      3.0
105733      105733  0.888788  1.228777      3.0
105734      105734  0.888249  1.231356      3.0
105735      105735  0.887806  1.234000      3.0
105736      105736  0.887462  1.236710      3.0
...            ...       ...       ...      ...
113196      113196  1.502762  0.820121      NaN
113197      113197  1.503860  0.822183      NaN
113198      113198  1.505077  0.824352      NaN
113199      113199  1.505716  0.826316      NaN
113200      113200  1.506881  0.828477      NaN

[7469 rows x 4 columns]


In [24]:
# CROP ONLY THE TRANSITION E.G. BETWEEN 0.5 - 1.0mA
this_sig = sm_toplot

fig, ax1 = plt.subplots(figsize = (8,6))
ax2 = ax1.twinx()

#ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['Spontan'], label = 'Med-Induced')
ax1.plot(np.arange(0, this_sig.shape[0]), this_sig['StimOn'], label = 'Subharmonic')
ax2.plot(np.arange(0, this_sig.shape[0]), this_sig['StimVec'], ls='--', color = 'grey')
ax1.axvline(x = 5000, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Stim Off')
#ax1.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)


ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax1.set_xlabel('Time [samples]')
ax1.set_title(str(subID))


Text(0.5, 1.0, 'Sub065')

In [25]:
fft_fig = os.path.join(project_path, 'figures','anal_signal', 'switch_off/')
fft_file = os.path.join(project_path, 'data','anal_signal', 'switch_off/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignalSWITCH_OFF',dpi = 150)
this_sig.to_csv(str(fft_file)+str(fft_name)+'sm_analSignalSWITCH_OFF.csv')

### Average Cropped Analytic Signal and Plot

In [ ]:
'''
med_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllMed_analSignalTRANSITION.csv'
))

stim_anal = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'AllStim_analSignalTRANSITION.csv'
))

print(stim_anal)
'''

med_anal = []
stim_anal = []

ps_path = os.path.join(
   project_path,
   'data', 'anal_signal'
)

for filename in os.listdir(ps_path):
   if filename.endswith('TRANSITION.csv'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
      this_df = pd.read_csv(os.path.join(ps_path,filename))
      this_med_anal = this_df['Spontan']
      this_stim_anal = this_df['StimOn']


      med_anal.append(this_med_anal)
      stim_anal.append(this_stim_anal)



In [ ]:
all_med_anal_df = pd.DataFrame(med_anal)
all_stim_anal_df = pd.DataFrame(stim_anal)
all_med_anal_df.shape

In [ ]:
med_anal_cr = all_med_anal_df.loc[:, 1:7500] #all_med_anal_df.loc[:, 7500:17500]
med_anal_mean = np.nanmean(med_anal_cr,0)
med_anal_sem = stats.sem(med_anal_cr,0, nan_policy='omit')

stim_anal_cr = all_stim_anal_df.loc[:,  1:7500]
stim_anal_mean = np.nanmean(stim_anal_cr,0)
stim_anal_sem = stats.sem(stim_anal_cr,0, nan_policy='omit')


In [ ]:
med_anal_cr


In [ ]:
2000/250

In [ ]:
%matplotlib qt
plt.plot(np.arange(0,7500), med_anal_mean, label = 'Med-Induced FTG') #10000
plt.fill_between(np.arange(0,7500), med_anal_mean-med_anal_sem, med_anal_mean+med_anal_sem, alpha = 0.2)

plt.plot(np.arange(0,7500), stim_anal_mean, color = 'red', label = 'Stim-Induced FTG')
plt.fill_between(np.arange(0,7500), stim_anal_mean-stim_anal_sem, stim_anal_mean+stim_anal_sem, color = 'red', alpha = 0.2)


#plt.axvline(x = 6250, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Subharmonic On')
#plt.axvline(x = 7500, color = 'grey', lw=3, alpha = 0.2)
#plt.xlim(0,10000)
#plt.xticks(np.arange(0,11250,1250), labels = np.arange(-25,20,5))
plt.xlim(0,2500)

plt.legend(loc = 'upper left')
plt.xlabel('Time [sec]')
plt.ylabel('Z-scored Smoothed Analytic Signal')

In [ ]:
fft_fig = os.path.join(project_path, 'results\\')

plt.savefig(str(fft_fig)+'AVG_AnalCropped_StimOn',dpi = 250)

In [ ]:
pval = 0.05  # arbitrary
dfn = 2 - 1  # degrees of freedom numerator
dfd = med_anal.shape[0] - 2  # degrees of freedom denominator
thresh = scipy.stats.f.ppf(1 - pval, dfn=dfn, dfd=dfd)  # F distribution
print(thresh)

In [ ]:
perm_nd = np.empty([6,med_anal.shape[0]])
perm_nd[:] = np.nan
perm_nd[0,:] = np.transpose(med_anal)

In [ ]:
y = np.expand_dims(med_anal, axis = 2)
y1 = np.expand_dims(stim_anal, axis = 2)


In [ ]:
#H0 = np.empty([1000,6])
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([y, y1], n_permutations=1000,
                             threshold=None, n_jobs=None)


In [ ]:
cluster_p_values

### Make all Signals same Length

In [ ]:
import traces


In [ ]:
sm_anal = pd.read_csv(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'Sub005_sm_analSignal.csv'
))

print(sm_anal)

subID = 'Sub005'
fft_name = str(subID) + '_'
print(fft_name)

In [ ]:
%matplotlib qt
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['Spontan'], label = 'Med Induced')
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimOn'], label = 'Stim Induced')
plt.plot(np.arange(0, sm_anal.shape[0]), sm_anal['StimVec'], ls='--', color = 'grey')
plt.legend()

In [ ]:
np.unique(np.round(sm_anal['StimVec'], decimals = 1))

In [ ]:
sm_anal['StimVec'] = np.round(sm_anal['StimVec'], decimals = 1)
this_anal = sm_anal
plt.plot(sm_anal['Spontan'][sm_anal['StimVec'] == 0])
plt.plot(sm_anal['StimOn'][sm_anal['StimVec'] == 0])
plt.plot(sm_anal['StimVec'][sm_anal['StimVec'] == 0])

In [ ]:
amp_of_interest = [0, 0.5, 1]

sm_anal['Spontan'][250:300,sm_anal['StimVec'] == 0]

### Switching Stim Off Analytic Signal for all

In [26]:
stim_anal = []

ps_path = os.path.join(
   project_path,
   'data', 'anal_signal', 'switch_off'
)

for filename in os.listdir(ps_path):
   if filename.endswith('SWITCH_OFF.csv'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
      this_df = pd.read_csv(os.path.join(ps_path,filename))
      this_stim_anal = this_df['StimOn']

      stim_anal.append(this_stim_anal)



In [31]:
all_stim_anal_df = pd.DataFrame(stim_anal)

mean_values = all_stim_anal_df.mean(skipna=True)
sem_values =  all_stim_anal_df.sem(skipna=True)


In [32]:

# Plot the mean with shaded area for 
plot_mean = mean_values[:]
plot_sem = sem_values[:]
plt.plot(plot_mean, color='blue', label = 'Stim-Induced FTG')
plt.fill_between(plot_mean.index, plot_mean - plot_sem, plot_mean + plot_sem, color='lightblue')
plt.axvline(x = 5000, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Stim Off')
plt.ylim(0.5,2)
plt.xlim(2500,7500)
plt.xticks(np.arange(2500,8000,500), labels=np.arange(-10,12,2))
plt.legend()
plt.xlabel('Time [sec]')
plt.ylabel('Z-scored Smoothed Analytic Signal')

Text(0, 0.5, 'Z-scored Smoothed Analytic Signal')

In [33]:
fft_fig = os.path.join(project_path, 'results\\')

plt.savefig(str(fft_fig)+'SWITCH_STIM_OFF',dpi = 250)